In [13]:
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import sys
import os
import pandas as pd 
import argparse
import random
from scipy.spatial.transform import Rotation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

# Imposta il seed per numpy
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

custom = True

_size = "500"
dataset = "h36m"
_type = "3D"
ceil = 10 
dim = 2
method = "gt"
name = "completion_murino_"+_type+"_"+dataset+"_" + method +"_mask"+str(ceil)+"_norot" # 0.0411 with +30*



def random_rotation_2d():    
    # Genera un angolo casuale per la rotazione in radianti
    #angle = np.random.uniform(0, 2 * np.pi)
    #angle = np.random.uniform(-np.pi/6,np.pi/6)
    angle=0
    # Crea la matrice di rotazione 2D
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],
                                [np.sin(angle), np.cos(angle)]])

    return rotation_matrix

def random_rotation_3d():    
    # Genera una rotazione casuale tramite l'uso della libreria `scipy`
    rotation = Rotation.from_euler('xyz', np.random.rand(3), degrees=False)
    
    # Estrai la matrice di rotazione dalla rotazione
    rotation_matrix = rotation.as_matrix()
    
    return rotation_matrix




model_name = '../models/' + name + '.h5'
input_path = "../local_dataset/training/completion/"+_type+"/"

if method is not None:
    X_train = np.load(os.path.join(input_path,"X_train_" + dataset + "_" +  _size +  "_minmax.npy")).swapaxes(1,2)
    X_val = np.load(os.path.join(input_path,"X_val_" + dataset + "_" +  _size +  "_minmax.npy")).swapaxes(1,2)
else:
    X_train = np.load(os.path.join(input_path,"X_train_" + dataset + "_" + method + "_" + _size +  "_minmax.npy")).swapaxes(1,2)
    X_val = np.load(os.path.join(input_path,"X_val_" + dataset + "_" + method + "_" + _size +  "_minmax.npy")).swapaxes(1,2)


if custom:
    X_train = np.load("/home/emartini/nas/MAEVE/HUMAN_MODEL/dataset/training_FLK2/h36m_cameraview_3D/X_train_1000.npy")
    X_val = np.load("/home/emartini/nas/MAEVE/HUMAN_MODEL/dataset/training_FLK2/h36m_cameraview_3D/X_val_1000.npy")
    model_name = "tmp/MLP_bio_h36m_cameraview_3D_absolute.h5"
    dim = 3

#X_val = np.load(os.path.join(input_path,"X_val_aist_" +  _size +  "_minmax.npy")).swapaxes(1,2)
#X_test = np.load(os.path.join(input_path,"X_test_" + dataset + "_" +  _size +  ".npy")).swapaxes(1,2)

#X_train = X_train.swapaxes(1,2)
#X_val = X_val.swapaxes(1,2)

print(X_train[0,:,:])
print(X_val[0,:,:])

[[-364.1775   -975.4356   6021.122    -109.1984   -954.0909   6099.669
  -508.9153   -730.9489   5921.708       6.785923 -680.884    6149.784
  -557.0898   -488.4602   5995.181     -46.69554  -470.2704   6288.714
  -372.7751   -575.7365   5872.574    -103.1269   -597.285    5963.012
  -446.7088   -132.9091   6060.143    -170.6657   -143.0198   6123.739
  -456.9418    323.8216   6125.481    -171.2247    317.4601   6093.172   ]]
[[  90.01911  -712.6233   3312.449      65.28832  -637.2011   3006.589
    12.47191  -532.5627   3516.78        3.935964 -372.9722   2925.318
   171.4691   -390.2531   3389.79      176.4971   -232.0712   3034.999
    34.38879  -260.741    3373.961     -61.70858  -226.4938   3117.111
    85.31891   195.8902   3421.741     -87.43157   221.6817   3226.042
    -5.04759   645.9555   3454.621    -365.6985    473.6731   3492.263   ]]


In [14]:
keypoints = int(X_train.shape[2]/dim) 

# X_train = X_train[:100,:,:]
# X_val = X_val[:100,:,:]

X_train_enhanced = np.repeat(X_train,1,axis=0)


In [65]:

def normalize_bio_root_relative(dataset):
    from Skeleton import Skeleton,ConstrainedSkeleton
    s12 = Skeleton('BODY12.xml')
    s15 = ConstrainedSkeleton('BODY15_constrained.xml')
    # Normalization using Biomecanichal model
    labels = ['LShoulder','RShoulder','LElbow','RElbow','LWrist','RWrist','LHip','RHip','LKnee','RKnee','LAnkle','RAnkle']
    
    for j in range(dataset.shape[0]):
        x = dataset[j,:,:].reshape(-1,3)
        s12.load_from_numpy(x,labels)
        s15.load_from_BODY12(s12)
        h = s15.estimate_height(False)
        s12.load_from_numpy(x/h,labels)
        s15.load_from_BODY12(s12)
        s15.relative_position()
        dataset[j,:,:] = s15.to_numpy(labels).ravel()

normalize_bio_root_relative(X_train_enhanced)
normalize_bio_root_relative(X_val)

In [15]:
def normalize_bio_root_absolute(dataset):
    from Skeleton import Skeleton,ConstrainedSkeleton
    s12 = Skeleton('BODY12.xml')
    s15 = ConstrainedSkeleton('BODY15_constrained.xml')
    # Normalization using Biomecanichal model
    labels = ['LShoulder','RShoulder','LElbow','RElbow','LWrist','RWrist','LHip','RHip','LKnee','RKnee','LAnkle','RAnkle']
    
    for j in range(dataset.shape[0]):
        x = dataset[j,:,:].reshape(-1,3)
        s12.load_from_numpy(x,labels)
        s15.load_from_BODY12(s12)
        h = s15.estimate_height(False)
        root = s15.to_numpy(["Root"])
        x = (x-root)/h
        dataset[j,:,:] = x.ravel()

normalize_bio_root_absolute(X_train_enhanced)
normalize_bio_root_absolute(X_val)

In [ ]:
def normalize_bio_root_absolute_oneauto(dataset):
    from Skeleton import Skeleton,ConstrainedSkeleton
    s12 = Skeleton('BODY12.xml')
    s15 = ConstrainedSkeleton('BODY15_constrained.xml')
    # Normalization using Biomecanichal model
    labels = ['LShoulder','RShoulder','LElbow','RElbow','LWrist','RWrist','LHip','RHip','LKnee','RKnee','LAnkle','RAnkle']
    
    for j in range(dataset.shape[0]):
        x = dataset[j,:,:].reshape(-1,3)
        s12.load_from_numpy(x,labels)
        s15.load_from_BODY12(s12)
        h = s15.estimate_height(False)
        root = s15.to_numpy(["Root"])
        x = (x-root)/h
        dataset[j,:,:] = x.ravel()

normalize_bio_root_absolute_oneauto(X_train_enhanced)
normalize_bio_root_absolute_oneauto(X_val)

In [16]:
# print("Rotating")
# for r in range(X_train_enhanced.shape[0]):
#         if dim==2:
#             RT = random_rotation_2d()
#         elif dim==3:
#             RT = random_rotation_3d()
#         X_train_rotated_start = X_train_enhanced[r,:].reshape((-1,dim))
#         X_train_rotated_end = X_train_rotated_start@RT
#         X_train_enhanced[r,:] = X_train_rotated_end.reshape((1,-1))

print("Masking")
y_train = X_train_enhanced.copy()
for r in range(X_train_enhanced.shape[0]):      
        kps_to_mask = np.random.choice(list(range(keypoints)),np.random.randint(low=1,high=ceil+1),replace=False)
        for j in kps_to_mask:
            X_train_enhanced[r,0,dim*j:dim*j+dim] = np.nan
X_train = X_train_enhanced.copy()


y_val = X_val.copy()

for r in range(X_val.shape[0]):      
        kps_to_mask = np.random.choice(list(range(keypoints)),np.random.randint(low=1,high=ceil+1),replace=False)
        for j in kps_to_mask:
            X_val[r,0,dim*j:dim*j+dim] = np.nan


Masking


#### Normalization minmax

In [37]:
# Normalization [min,max]->[0,1]

def normalize_minmax(dataset,gt):
    for j in range(dataset.shape[0]):
        skeleton = dataset[j,:,:]#.reshape(-1,3)
        #print(skeleton)
        #break
        print(skeleton)
        in_max = np.nanmax(skeleton)
        in_min = np.nanmin(skeleton)
        dataset[j,:,:] = (skeleton.copy()-in_min)/(in_max-in_min)
        gt[j,:,:] = (gt[j,:,:].copy()-in_min)/(in_max-in_min)
        print(skeleton)
        

normalize_minmax(X_train,y_train)
normalize_minmax(X_val,y_val)


[[-364.1775   -975.4356   6021.122    -109.1984   -954.0909   6099.669
  -508.9153   -730.9489   5921.708       6.785923 -680.884    6149.784
  -557.0898   -488.4602   5995.181     -46.69554  -470.2704   6288.714
  -372.7751   -575.7365   5872.574    -103.1269   -597.285    5963.012
  -446.7088   -132.9091   6060.143    -170.6657   -143.0198   6123.739
  -456.9418    323.8216   6125.481    -171.2247    317.4601   6093.172   ]]
[[0.08414723 0.         0.96316265 0.11924826 0.00293836 0.97397562
  0.06422229 0.03365662 0.94947709 0.13521494 0.04054867 0.98087457
  0.05759047 0.06703818 0.95959155 0.12785255 0.06954224 1.
  0.08296367 0.05502352 0.94271318 0.12008408 0.05205711 0.95516309
  0.07278578 0.11598419 0.96853438 0.11078653 0.11459233 0.97728915
  0.07137708 0.17885882 0.97752896 0.11070957 0.17798308 0.97308123]]
[[ 108.2874  -548.9797  5065.423   -102.9111  -668.8245  5191.994
    65.87762 -255.134   5015.804   -389.7086  -683.654   5282.177
  -148.5072  -112.4352  5029.806   

In [17]:
# Creazione del modello Masked Autoencoder
@keras.saving.register_keras_serializable('MLP')
class MLP(tf.keras.Model):
    def __init__(self):
        super(MLP, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(input_shape=keypoints * dim)
        self.layer1 = tf.keras.layers.Dense(128, activation='relu')
        self.layer2 = tf.keras.layers.Dense(256, activation='relu')
        self.layer3 = tf.keras.layers.Dense(128, activation='relu')
        self.layer4 = tf.keras.layers.Dense(keypoints * dim, activation='linear')

    def build(self, input_shape):
        # Definisci i pesi dei layer in base alle dimensioni dell'input
        super(MLP, self).build(input_shape)

    def call(self, inputs):
  
        x = tf.where(tf.math.is_nan(inputs), -1 * tf.ones_like(inputs), inputs)
        x = self.input_layer(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        decoded = self.layer4(x)
        return decoded

def custom_mse_loss(y_true, y_pred):
    mse_class = tf.reduce_mean(tf.square(y_true - y_pred))
    return mse_class


autoencoder = MLP()
autoencoder.build(input_shape=(1, keypoints * dim))
autoencoder.summary()



autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=custom_mse_loss, metrics=["mae"])
history = autoencoder.fit(X_train.squeeze(), y_train.squeeze(), batch_size=128, epochs=100, shuffle = True, validation_data=(X_val,y_val))
autoencoder.save_weights(model_name)

# opt = keras.optimizers.Adam(learning_rate=1e-5)
# autoencoder.compile(optimizer=opt, loss=custom_mse_loss, metrics=["mae"])
# batch_size = 128
# epochs = 100
# for epoch in range(epochs):
#     print("epoch",epoch)
#     indices = np.arange(len(X_train))
#     np.random.shuffle(indices)
#     X_train_shuffled = X_train[indices]
#     y_train_shuffled = y_train[indices]
#     for i in range(0, len(X_train_shuffled), batch_size):
#         X_batch = X_train_shuffled[i:i+batch_size]
#         y_batch = y_train_shuffled[i:i+batch_size]
#         autoencoder.train_on_batch(X_batch, y_batch)

#autoencoder.save_weights(model_name)
# autoencoder.save_weights(model_name)

Model: "mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 36)]              0         
                                                                 
 dense (Dense)               multiple                  4736      
                                                                 
 dense_1 (Dense)             multiple                  33024     
                                                                 
 dense_2 (Dense)             multiple                  32896     
                                                                 
 dense_3 (Dense)             multiple                  4644      
                                                                 
Total params: 75300 (294.14 KB)
Trainable params: 75300 (294.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-12-19 12:12:58.574197: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-19 12:12:58.574343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-19 12:12:58.574427: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/100


2023-12-19 12:12:59.654854: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f484cb3af80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-19 12:12:59.654876: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2023-12-19 12:12:59.658599: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-19 12:12:59.672451: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-19 12:12:59.720003: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5250/5250 [==============================] - 10s 2ms/step - loss: 0.0034 - mae: 0.0375 - val_loss: 0.0025 - val_mae: 0.0313
Epoch 2/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0022 - mae: 0.0288 - val_loss: 0.0021 - val_mae: 0.0271
Epoch 3/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0021 - mae: 0.0272 - val_loss: 0.0021 - val_mae: 0.0275
Epoch 4/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0020 - mae: 0.0265 - val_loss: 0.0021 - val_mae: 0.0271
Epoch 5/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0020 - mae: 0.0260 - val_loss: 0.0019 - val_mae: 0.0260
Epoch 6/100
5250/5250 [==============================] - 8s 2ms/step - loss: 0.0019 - mae: 0.0256 - val_loss: 0.0019 - val_mae: 0.0249
Epoch 7/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0019 - mae: 0.0253 - val_loss: 0.0019 - val_mae: 0.0257
Epoch 8/100
5250/5250 [==============================] - 9s 2ms/st

In [18]:
autoencoder.summary()
autoencoder.save_weights(model_name)
autoencoder.save(model_name.replace(".h5",""))

Model: "mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 36)]              0         
                                                                 
 dense (Dense)               multiple                  4736      
                                                                 
 dense_1 (Dense)             multiple                  33024     
                                                                 
 dense_2 (Dense)             multiple                  32896     
                                                                 
 dense_3 (Dense)             multiple                  4644      
                                                                 
Total params: 75300 (294.14 KB)
Trainable params: 75300 (294.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
INFO:tensorflow:Assets wr

INFO:tensorflow:Assets written to: tmp/MLP_bio_h36m_cameraview_3D_absolute/assets
